## 1. Tải dữ liệu

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/protonx-cloud-storage/data.txt
data = open('data.txt').read()

--2022-06-25 13:38:00--  https://storage.googleapis.com/protonx-cloud-storage/data.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.128, 74.125.20.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘data.txt’

data.txt            100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2022-06-25 13:38:00 (88.3 MB/s) - ‘data.txt’ saved [93578/93578]



## 2. Thêm các thư viện cần thiết

In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np 

## 3. Xử lý dữ liệu

In [3]:
corpus = data.lower().split("\n")

In [4]:
corpus[:4]

['from fairest creatures we desire increase,',
 "that thereby beauty's rose might never die,",
 'but as the riper should by time decease,',
 'his tender heir might bear his memory:']

### 3.1. Xây dựng tokenizer

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

### 3.2. Tách câu

Tách từng câu thành nhiều phần có chiều dài tăng dần để làm từng điểm dữ liệu

In [6]:
input_sequences = []
for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

In [7]:
input_sequences[:10]

[[34, 417],
 [34, 417, 877],
 [34, 417, 877, 166],
 [34, 417, 877, 166, 213],
 [34, 417, 877, 166, 213, 517],
 [8, 878],
 [8, 878, 134],
 [8, 878, 134, 351],
 [8, 878, 134, 351, 102],
 [8, 878, 134, 351, 102, 156]]

In [8]:
tokenizer.sequences_to_texts([[34, 417]])

['from fairest']

In [9]:
for point in input_sequences[:10]:
  print(" ".join(tokenizer.sequences_to_texts([point])))

from fairest
from fairest creatures
from fairest creatures we
from fairest creatures we desire
from fairest creatures we desire increase
that thereby
that thereby beauty's
that thereby beauty's rose
that thereby beauty's rose might
that thereby beauty's rose might never


### 3.3. Chia features, label

Thực hiện Padding

In [10]:
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

Các câu được tách bên trên sẽ được padding với việc chèn các giá trị 0 vào trước câu để tạo ra các câu có chiều dài bằng nhau.

Việc **padding vào trước** vì bài toán này ta sẽ sinh từ từ phía sau nên thông tin bên phải là những giá trị khác 0. 

In [11]:
input_sequences[:10]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,  34, 417],
       [  0,   0,   0,   0,   0,   0,   0,   0,  34, 417, 877],
       [  0,   0,   0,   0,   0,   0,   0,  34, 417, 877, 166],
       [  0,   0,   0,   0,   0,   0,  34, 417, 877, 166, 213],
       [  0,   0,   0,   0,   0,  34, 417, 877, 166, 213, 517],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   8, 878],
       [  0,   0,   0,   0,   0,   0,   0,   0,   8, 878, 134],
       [  0,   0,   0,   0,   0,   0,   0,   8, 878, 134, 351],
       [  0,   0,   0,   0,   0,   0,   8, 878, 134, 351, 102],
       [  0,   0,   0,   0,   0,   8, 878, 134, 351, 102, 156]],
      dtype=int32)

Cắt cột cuối cùng của ma trận bên trên để làm nhãn

In [12]:
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
# Chuyển thành One Hot vector


In [13]:
label[i:i+1]

array([351], dtype=int32)

In [14]:
for i in range(10):
  print("{} ---> {}".format(" ".join(tokenizer.sequences_to_texts(predictors[i:i+1])), tokenizer.sequences_to_texts([label[i:i+1]])[0]))

from ---> fairest
from fairest ---> creatures
from fairest creatures ---> we
from fairest creatures we ---> desire
from fairest creatures we desire ---> increase
that ---> thereby
that thereby ---> beauty's
that thereby beauty's ---> rose
that thereby beauty's rose ---> might
that thereby beauty's rose might ---> never


Chuyển từng nhãn thành vector one hot với số lượng là số từ trong từ điển

In [15]:
label = ku.to_categorical(label, num_classes=total_words)

## 4. Xây dựng model

Mạng bao gồm:

- 1 lớp Embedding với chiều embedding là 100
- Một lớp 1 Bidirectional với cell LSTM 150 node
- Một lớp LSTM với 100 node
- Mạng nơ ron phân loại gồm một lớp ẩn


In [16]:
model = Sequential()

model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))

model.add(Bidirectional(LSTM(150, return_sequences = True)))

model.add(Dropout(0.2))

model.add(LSTM(100))

model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 100)           321100    
                                                                 
 bidirectional (Bidirectiona  (None, 10, 300)          301200    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 10, 300)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               160400    
                                                                 
 dense (Dense)               (None, 1605)              162105    
                                                                 
 dense_1 (Dense)             (None, 3211)              5156866   
                                                        

Tiến hành training

In [17]:
 history = model.fit(predictors, label, epochs=130, verbose=0)

### 5. Dự đoán 10 từ tiếp theo

Câu mồi

In [18]:
test_seq = 'despite of wrinkles'

Từ câu mồi
- Dự đoán ra từ tiếp theo từ các từ của câu hiện tại
- Nối từ đã được dự đoán vào câu hiện tại
- Tiếp tục dùng câu hiện tại này để dự đoán
- Thực hiện đến khi chiều dài của câu đạt một giới hạn cụ thể hoặc số từ sinh ra đạt giới hạn

In [19]:
next_words = 10

for _ in range(next_words):
  # Chuyển câu thành vector
  token_list = tokenizer.texts_to_sequences([test_seq])[0]
  
  # Padding câu
  token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
  
  # Dự đoán từ tiếp theo
  predicted = model.predict(token_list, verbose=0)
  
  output_word = ""

  predicted_id = np.argmax(predicted)

  if predicted_id in tokenizer.index_word:
    output_word = tokenizer.index_word[predicted_id]
    test_seq += " " + output_word
  else:
    break
  
print(test_seq)


despite of wrinkles this thy golden time being hell of hide time being
